In [1]:
try:
    %load_ext iminizinc
except:
    !pip install iminizinc
    %load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.4.3, build 122680178
Copyright (C) 2014-2020 Monash University, NICTA, Data61


In [2]:
import os
import numpy as np
import pandas as pd

In [1]:
classes = 14
sections = 2
teachers = 7

### Randomly initialization

In [33]:
tList = np.random.randint(1,teachers+1,size=(classes))
sList = np.random.randint(1,sections+1,size=(classes))
hList = np.random.randint(3,4,size=(classes))

tList[0] = 1
j=0
for i in range(0,classes):
    if i%sections==0:
#         print("i ran",tList[i])
        j=j+1
        for s in range(0,sections):
            tList[i+s] = j
            sList[i+s] = s+1

classDetails = np.array([tList, sList, hList])
classDetails = classDetails.T
# print(classDetails)
# print(classDetails.shape[0])
print(classDetails)
classes = classDetails.shape[0]
classDetails = classDetails.tolist()

[[1 1 3]
 [1 2 3]
 [2 1 3]
 [2 2 3]
 [3 1 3]
 [3 2 3]
 [4 1 3]
 [4 2 3]
 [5 1 3]
 [5 2 3]
 [6 1 3]
 [6 2 3]
 [7 1 3]
 [7 2 3]]


In [34]:
tBusy = np.zeros((teachers,5,6))
for i in range(0,6):
    tBusy[0,0,i] = 1
    tBusy[0,1,i] = 1
    tBusy[1,0,i] = 1
# print(tBusy.astype(int))
teacherBusy = tBusy.astype(int).tolist()

In [35]:
slots = 6
days = 5
rooms = 2

In [36]:
roomRoutine = [[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]]

### Minizinc Model

In [41]:
%%minizinc -m bind

include "globals.mzn";
include "alldifferent.mzn";
int: days;
int: slots;

set of int: daysPerWeek = 1..days;
set of int: slotsPerDay = 1..slots;

set of int: properties = 1..3; 
%property0 = teacher
%property1 = section
%property2 = hours per week

int: classes;
int: rooms;
int: sections;
int: teachers;

set of int: sectionList = 1..sections;
set of int: teacherList = 1..teachers;
set of int: roomList = 1..rooms;
set of int: classList = 1..classes;
set of int: classIndex = 0..classes;
set of int: periods = 1..3;

array[classList,properties] of int: classDetails;

% array[teacherList,daysPerWeek,slotsPerDay] of var classIndex: teacherRoutine;
array[teacherList,daysPerWeek,slotsPerDay] of 0..1: teacherBusy;
array[sectionList,daysPerWeek,slotsPerDay] of var classIndex: sectionRoutine;
array[roomList,daysPerWeek,slotsPerDay] of var classIndex: roomRoutine;

set of int: allSlots = 1..rooms*days*slots;
array[classList,periods] of var allSlots: assignedSlots;
array[teacherList,1..6] of var allSlots: assignedTeachers;


constraint alldifferent(assignedSlots);

constraint
forall(i,j in classList,p in periods)
    (
        if i != j /\ classDetails[i,1] == classDetails[j,1]
            then getExcess(assignedSlot[i,1]) != getExcess
    );

% constraint 
% forall(i in classList,p in periods)
%     (
%         teacherRoutine[classDetails[i,1],getDay(assignedSlots[i,p]),getSlot(assignedSlots[i,p])] = i
%     );
    
% constraint 
% forall(i in classList)
%     (
%         sum(d in daysPerWeek)(count(teacherRoutine[classDetails[i,1],d,..],i)) == 3
%     );
    
% % matching room's routine with teachers'
% constraint 
% forall(t in teacherList,d in daysPerWeek, s in slotsPerDay)
%     (
%         if teacherRoutine[t,d,s] > 0
%         then teacherRoutine[t,d,s] == sectionRoutine[classDetails[teacherRoutine[t,d,s],2],d,s] 
%         else true 
%         endif
%     );
    
constraint
forall(i in classList)
    (
        getDay(assignedSlots[i,1]) < getDay(assignedSlots[i,2]) 
        /\ 
        getDay(assignedSlots[i,2]) < getDay(assignedSlots[i,3])
    );
    
% constraint 
% forall(i in classList,p in periods)
%     (
%         roomRoutine[getRoom(assignedSlots[i,p]),getDay(assignedSlots[i,p]),getSlot(assignedSlots[i,p])] = i
%     );

function var int: getSlot(var int: x) = if x mod slots == 0 then slots else x mod slots endif;

function var int: getDay(var int: x) = ((x - getSlot(x)) mod days) + 1 ;

function var int: getRoom(var int: x) = (x - getSlot(x) - (getDay(x)-1)*slots) div (days*slots) + 1;

function var int: getExcess(var int: x) = x - (getRoom(x)-1)*days*slots;

% var int: newMe;
% var int: newDay;
% var int: newRoom;
% constraint
% newMe = getSlot(assignedSlots[3,3]);
% constraint
% newDay = getDay(assignedSlots[3,3]);
% constraint
% newRoom = getRoom(assignedSlots[3,3]);
% constraint
% assignedSlots[3,3] == 120;


solve satisfy;

C:\Users\USER\AppData\Local\Temp\tmpxzhoe_0b/model.mzn:44.6:

     ^
Error: syntax error, unexpected ';', expecting else or elseif


In [40]:
%%minizinc -m bind

include "globals.mzn";
include "alldifferent.mzn";
int: days;
int: slots;

set of int: daysPerWeek = 1..days;
set of int: slotsPerDay = 1..slots;

set of int: properties = 1..3; 
%property0 = teacher
%property1 = section
%property2 = hours per week

int: classes;
int: rooms;
int: sections;
int: teachers;

set of int: sectionList = 1..sections;
set of int: teacherList = 1..teachers;
set of int: roomList = 1..rooms;
set of int: classList = 1..classes;
set of int: classIndex = 0..classes;
set of int: periods = 1..3;

array[classList,properties] of int: classDetails;

array[teacherList,daysPerWeek,slotsPerDay] of var classIndex: teacherRoutine;
array[teacherList,daysPerWeek,slotsPerDay] of 0..1: teacherBusy;
array[sectionList,daysPerWeek,slotsPerDay] of var classIndex: sectionRoutine;
array[roomList,daysPerWeek,slotsPerDay] of var classIndex: roomRoutine;

set of int: allSlots = 1..rooms*days*slots;
array[classList,periods] of allSlots: assignedSlots;

constraint alldifferent(assignedSlots);

constraint 
forall(i in classList,p in periods)
    (
        teacherRoutine[classDetails[i,1],getDay(assignedSlots[i,p]),getSlot(assignedSlots[i,p])] = i
    );

constraint 
forall(i in classList,p in periods)
    (
        sectionRoutine[classDetails[i,2],getDay(assignedSlots[i,p]),getSlot(assignedSlots[i,p])] = i
    );

constraint 
forall(i in classList,p in periods)
    (
        roomRoutine[getRoom(assignedSlots[i,p]),getDay(assignedSlots[i,p]),getSlot(assignedSlots[i,p])] = i
    );

% constraint 
% forall(i in classList)
%     (
%         sum(d in daysPerWeek)(count(teacherRoutine[classDetails[i,1],d,..],i)) == 3
%     );
    
% % matching room's routine with teachers'
% constraint 
% forall(t in teacherList,d in daysPerWeek, s in slotsPerDay)
%     (
%         if teacherRoutine[t,d,s] > 0
%         then teacherRoutine[t,d,s] == sectionRoutine[classDetails[teacherRoutine[t,d,s],2],d,s] 
%         else true 
%         endif
%     );
    

    

function var int: getSlot(var int: x) = if x mod slots == 0 then slots else x mod slots endif;

function var int: getDay(var int: x) = ((x - getSlot(x)) mod days) + 1 ;

function var int: getRoom(var int: x) = (x - getSlot(x) - (getDay(x)-1)*slots) div (days*slots) + 1;


solve satisfy;


  C:\Users\USER\AppData\Local\Temp\tmpcy3lg91e/model.mzn:46:
  in call 'forall'
  in array comprehension expression
    with i = 7
    with p = 3
  C:\Users\USER\AppData\Local\Temp\tmpcy3lg91e/model.mzn:48:
  in binary '=' operator expression
Solver output:
The problem is infeasible
No solutions found


### Print Output

In [28]:
print(assignedSlots)

[[36, 42, 24], [35, 41, 23], [34, 40, 22], [33, 39, 21], [32, 38, 20], [31, 37, 19], [6, 12, 18], [5, 11, 17], [4, 10, 16], [3, 9, 15], [2, 8, 14], [1, 7, 13]]


In [29]:
print(np.array(teacherRoutine))

[[[ 0  0  0  0  2  1]
  [ 0  0  0  0  2  1]
  [ 0  0  0  0  0  0]
  [ 0  0  0  0  2  1]
  [ 0  0  0  0  0  0]]

 [[ 0  0  4  3  0  0]
  [ 0  0  4  3  0  0]
  [ 0  0  0  0  0  0]
  [ 0  0  4  3  0  0]
  [ 0  0  0  0  0  0]]

 [[ 6  5  0  0  0  0]
  [ 6  5  0  0  0  0]
  [ 0  0  0  0  0  0]
  [ 6  5  0  0  0  0]
  [ 0  0  0  0  0  0]]

 [[ 0  0  0  0  8  7]
  [ 0  0  0  0  8  7]
  [ 0  0  0  0  8  7]
  [ 0  0  0  0  0  0]
  [ 0  0  0  0  0  0]]

 [[ 0  0 10  9  0  0]
  [ 0  0 10  9  0  0]
  [ 0  0 10  9  0  0]
  [ 0  0  0  0  0  0]
  [ 0  0  0  0  0  0]]

 [[12 11  0  0  0  0]
  [12 11  0  0  0  0]
  [12 11  0  0  0  0]
  [ 0  0  0  0  0  0]
  [ 0  0  0  0  0  0]]]


In [24]:
sectionRoutine

[[[0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]]]

In [25]:
roomRoutine

[[[12, 11, 10, 9, 8, 7],
  [12, 11, 10, 9, 8, 7],
  [12, 11, 10, 9, 8, 7],
  [6, 5, 4, 3, 2, 1],
  [6, 5, 4, 3, 2, 1]],
 [[6, 5, 4, 3, 2, 1],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]]]